In [ ]:
'''Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
# Summary of the algorithm
- We start with input sequences from a domain (e.g. English sentences)
    and correspding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and 丢弃 the outputs).
    保留状态作为decoder的上下文
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
    通过训练，把整个时间序列变成向后偏移了一个时间步长的同一序列
    
    
    以下是生成的流程，上边是训练
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors 第一步把输入数据作为状态向量
    - Start with a target sequence of size 1       从长度1开始
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence   把状态向量和一个字符放进去decode出下一个字符
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).                            使用argmax采样出下一个字符
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we   重复操作直到句子长度限制
        hit the character limit.
# Data download
English to French sentence pairs.
http://www.manythings.org/anki/fra-eng.zip
Lots of neat sentence pairs datasets can be found at:
http://www.manythings.org/anki/
# References
- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078
1) Turn the sentences into 3 Numpy arrays, encoder_input_data, decoder_input_data, decoder_target_data:
 1.encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters)
containing a one-hot vectorization of the English sentences.

 2.decoder_input_data is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters)
containg a one-hot vectorization of the French sentences.

 3.decoder_target_data is the same as decoder_input_data but offset by one timestep. 
decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :].

由于训练过程和推理过程（译码句）是完全不同的，所以我们要使用不同的模型，尽管它们都是利用相同的内部层。



In [1]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

/Users/houzhuo/Documents/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [22]:
batch_size = 64  # Batch size for training.
epochs = 25  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'cmn-eng/cmn.txt'

In [9]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')#['Hi.\t嗨。', 'Hi.\t你好。', 'Run.\t你用跑的。']
for line in lines[: min(num_samples, len(lines) - 1)]:
    #print(line)#Hi.  嗨\n Hi. 你好
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:#构建词集
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
    

In [12]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])#文本最大长度

print(input_characters)
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

[' ', '!', '"', '$', '%', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '’']
Number of samples: 10000
Number of unique input tokens: 73
Number of unique output tokens: 2630
Max sequence length for inputs: 31
Max sequence length for outputs: 22


In [15]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
print(input_token_index)
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
print(encoder_input_data.shape)

{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, '?': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'W': 43, 'Y': 44, 'Z': 45, 'a': 46, 'b': 47, 'c': 48, 'd': 49, 'e': 50, 'f': 51, 'g': 52, 'h': 53, 'i': 54, 'j': 55, 'k': 56, 'l': 57, 'm': 58, 'n': 59, 'o': 60, 'p': 61, 'q': 62, 'r': 63, 's': 64, 't': 65, 'u': 66, 'v': 67, 'w': 68, 'x': 69, 'y': 70, 'z': 71, '’': 72}
(10000, 31, 73)


In [19]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
        #num_pairs, max_english_sentence_length, num_english_characters,输入矩阵的相应位置置1
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.#很关键的teacher forcing


In [20]:
'''参数：
return_state contructor参数，配置一个RNN层返回第一个条目是输出，下一个条目是内部RNN状态的列表。用于恢复encoder的状态。

inital_state参数，指定RNN的初始状态。用于将encoder状态传递到decoder作为初始状态。

The return_sequences constructor argument,
configuring a RNN to return its full sequence of outputs(instead of just the last output, which the defaults behavior？？).
return_sequences构造函数参数，配置RNN返回整个输出序列。在decoder中使用
'''
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))#shape是（none，encoder的词袋长度），还是说none为主体，后边fit时候可替换？
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [23]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,#fit的时候采用的是前面的时序序列
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')#返回最好的结果


Train on 8000 samples, validate on 2000 samples
Epoch 1/25
8000/8000 [==============================] - 249s 31ms/step - loss: 1.9619 - val_loss: 2.4675
Epoch 2/25
8000/8000 [==============================] - 245s 31ms/step - loss: 1.8579 - val_loss: 2.3867
Epoch 3/25
8000/8000 [==============================] - 249s 31ms/step - loss: 1.7583 - val_loss: 2.2869
Epoch 4/25
8000/8000 [==============================] - 242s 30ms/step - loss: 1.6759 - val_loss: 2.2149
Epoch 5/25
8000/8000 [==============================] - 242s 30ms/step - loss: 1.5963 - val_loss: 2.1288
Epoch 6/25
8000/8000 [==============================] - 241s 30ms/step - loss: 1.5259 - val_loss: 2.0986
Epoch 7/25
8000/8000 [==============================] - 246s 31ms/step - loss: 1.4619 - val_loss: 2.0420
Epoch 8/25
8000/8000 [==============================] - 272s 34ms/step - loss: 1.4048 - val_loss: 1.9839
Epoch 9/25
8000/8000 [==============================] - 257s 32ms/step - loss: 1.3542 - val_loss: 1.9460
Epoch 1

KeyboardInterrupt: 

In [25]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state获取初始状态
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [26]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))#重置target_seg，把刚才产生的字符更新到target_seq
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence



In [27]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: 别要开着。

-
Input sentence: Hi.
Decoded sentence: 别要开着。

-
Input sentence: Run.
Decoded sentence: 把我们的车给我们。

-
Input sentence: Wait!
Decoded sentence: 把我们在家儿。

-
Input sentence: Hello!
Decoded sentence: 这是一个好的吗？

-
Input sentence: I try.
Decoded sentence: 不要说话。

-
Input sentence: I won!
Decoded sentence: 我喜欢你的。

-
Input sentence: Oh no!
Decoded sentence: 那是一個好的。

-
Input sentence: Cheers!
Decoded sentence: 它是一个好的。

-
Input sentence: He ran.
Decoded sentence: 他們有一個人。

-
Input sentence: Hop in.
Decoded sentence: 没有人。

-
Input sentence: I lost.
Decoded sentence: 我喜欢你的。

-
Input sentence: I quit.
Decoded sentence: 我喜欢你的。

-
Input sentence: I'm OK.
Decoded sentence: 我喜欢你的。

-
Input sentence: Listen.
Decoded sentence: 把我们的车给我们一样。

-
Input sentence: No way!
Decoded sentence: 没有人。

-
Input sentence: No way!
Decoded sentence: 没有人。

-
Input sentence: Really?
Decoded sentence: 你要在这里吗？

-
Input sentence: Try it.
Decoded sentence: 汤姆是个好人吗？

-
Input sentence: We 

c                




¥¥¥]¥¥

-0p[;'/.][
            'uklj,efdvcd21e3e12w2w`1q1q2ew2asasAZOLPPLLL877IU65T45R3E2W=-[99'[//
            '
            / 